第五章作业
编程实现
1. 例5.3，ID3算法
2. 例5.3，c4.5算法
3. 例5.4，cart分类树
4. 习题5.2 cart回归树

# 1.例5.3，ID3算法
参考代码：https://blog.csdn.net/GrinAndBearIt/article/details/79094079
把很多用python的操作重构成了numpy函数

In [149]:
import numpy as np
dataset = np.array([['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否']])
labels = ['年龄', '有工作', '有自己的房子', '信贷情况']

In [139]:
# 计算训练集的信息熵
import math
def calcInfoEntropy(dataset):
    unique_labels = np.unique(dataset[:,-1])
    sum_entropy = 0
    for i in range(len(unique_labels)):
        p = sum(dataset[:,-1]==unique_labels[i]) / len(dataset)
        sum_entropy -= p*math.log(p, 2)
#     print(f"训练集的信息熵：{sum_entropy}")
    return sum_entropy

In [140]:
# 根据某个特征划分数据集
# 即截取dataset中第i个特征为value的数据
def splitDataSet(dataset, i, value):
    tmp = dataset[dataset[:,i]==value]
    if i==0:
        return tmp[:,1:]
    elif i==dataset.shape[1]-1:
        return tmp[:,:i]
    else:
        return np.hstack((tmp[:,:i],tmp[:,i+1:]))

In [141]:
# 计算根据某个特征划分后的信息增益
def calcDivided(dataset, i):
    base_entropy = calcInfoEntropy(dataset)
    unique_values = np.unique(dataset[:,i])
    new_entropy = 0
    for value in unique_values:
        divided_dataset = splitDataSet(dataset, i, value)
        new_entropy += (len(divided_dataset)/dataset.shape[0])*calcInfoEntropy(divided_dataset)
    print(f"信息增益{base_entropy - new_entropy}")
    return base_entropy - new_entropy

In [142]:
# 寻找最大信息增益的特征
def findBestFeatures(dataset):
    m, n = dataset.shape
    bestInfoDivided = 0
    bestValue = 0
    for i in range(n-1):
        curInfoDivided = calcDivided(dataset, i)
        if curInfoDivided > bestInfoDivided:
            bestInfoDivided = curInfoDivided
            bestValue = i
    return bestValue

In [143]:
def maxResult(resultList):
    calcNumDict=dict([(np.sum(resultList==result),result) for result in resultList])
    return calcNumDict[max(calcNumDict.keys())]

In [144]:
def createTree(dataset, labels):
    resultList = dataset[:,-1]
    if dataset.shape[0] == 1:    #如果只剩一个元素
        return maxResult(resultList)
    if sum(resultList==resultList[0]) == resultList.shape[0]:   #如果剩下的全属于同一类
        return resultList[0]
    bestFeature = findBestFeatures(dataset)
#     print(f"bestFeature:{bestFeature}")
    bestLabel = labels[bestFeature]
    tree = {bestLabel:{}}
    print(f"最佳特征:{bestLabel}")
    del(labels[bestFeature])
    uniqueValues = set([data[bestFeature] for data in dataset])
    for value in uniqueValues:
        returnDataSet = splitDataSet(dataset, bestFeature, value)
        subLabels = labels[:]
        subTree = createTree(returnDataSet, subLabels)
        tree[bestLabel][value] = subTree
    return tree

In [146]:
print(createTree(dataset, labels))

信息增益0.08300749985576883
信息增益0.32365019815155627
信息增益0.4199730940219749
信息增益0.36298956253708536
最佳特征:有自己的房子
信息增益0.2516291673878229
信息增益0.9182958340544896
信息增益0.47385138961004514
最佳特征:有工作
{'有自己的房子': {'否': {'有工作': {'否': '否', '是': '是'}}, '是': '是'}}


# 2.例5.3，c4.5算法

In [147]:
# 将信息增益换成信息增益比
def calcDivided(dataset, i):
    base_entropy = calcInfoEntropy(dataset)
    unique_values = np.unique(dataset[:,i])
    new_entropy = 0
    for value in unique_values:
        divided_dataset = splitDataSet(dataset, i, value)
        new_entropy += (len(divided_dataset)/dataset.shape[0])*calcInfoEntropy(divided_dataset)
    print(f"信息增益{new_entropy/base_entropy}")
    return new_entropy/base_entropy

In [150]:
print(createTree(dataset, labels))

信息增益0.9145090385341493
信息增益0.6666666666666666
信息增益0.5674619322336875
信息增益0.6261503266899411
最佳特征:年龄
信息增益0.0
信息增益0.6684402927131713
信息增益0.979429340549307
最佳特征:信贷情况
信息增益0.0
信息增益0.0
最佳特征:有工作
信息增益0.0
信息增益1.0
最佳特征:有自己的房子
信息增益0.0
最佳特征:有工作
信息增益0.7632027404594346
信息增益0.7632027404594346
信息增益0.0
最佳特征:有工作
信息增益0.0
信息增益0.0
最佳特征:有自己的房子
信息增益0.8239348166312392
信息增益0.0
信息增益0.4119674083156196
最佳特征:有工作
信息增益0.0
信息增益0.0
最佳特征:有自己的房子
{'年龄': {'青年': {'信贷情况': {'一般': {'有工作': {'否': '否', '是': '是'}}, '好': {'有自己的房子': {'否': {'有工作': {'否': '否', '是': '是'}}}}}}, '老年': {'有工作': {'否': {'有自己的房子': {'是': '是', '否': '否'}}, '是': '是'}}, '中年': {'有工作': {'否': {'有自己的房子': {'否': '否', '是': '是'}}, '是': '是'}}}}


# 3.例5.4，cart分类树